# Notebook #3: Queries, Visualización y Análisis

- En este tercer notebook haremos consultas sobre nuestra base de datos y las usaremos para generar visualizaciones y el análisis de las mismas.

- El primer paso será importar las librerías necesarias:

In [78]:
# Librerías para tratamiento de datos
import pandas as pd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los resultado3s
import numpy as np

# Trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.

# Librerías para graficar
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

### Objetivo del Análisis 

- Antes de plantearnos las queries que pretendemos realizar, debemos recordar los objetivos del proyecto, así como la estructura de los datos.

    **1. Comparación de Precios entre Supermercados:** Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

    **2. Análisis de la Evolución de Precios:** Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.

    **3. Detección de Anomalías:** Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

    **4. Análisis de la Dispersión de Precios:** Evaluar la variabilidad de los precios de un mismo producto en diferentes supermercados.

    **5. Comparación de Precios Promedio:** Calcular y comparar los precios promedio de cada producto en diferentes supermercados.

#### Diagrama Entidad Relación

<img src="../images/Diagrama_ER.png" width="300">

### 1. Comparación de Precios entre Supermercados

In [2]:
query1 = '''
SELECT 
    s.nombre AS supermercado,
    c.nombre AS categoria,
    MIN(h.precioeur) AS precio_minimo,
    MAX(h.precioeur) AS precio_maximo
FROM 
    historico h
JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    s.nombre, c.nombre
ORDER BY 
    c.nombre, precio_minimo;
'''

In [40]:
resultado1 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query1)
resultado1

,supermercado,categoria,precio_minimo,precio_maximo
0,Alcampo,Aceite-de-girasol,1.29,6.92
1,Hipercor,Aceite-de-girasol,1.4,6.91
2,Eroski,Aceite-de-girasol,1.45,7.28
3,Carrefour,Aceite-de-girasol,1.45,40.67
4,Mercadona,Aceite-de-girasol,1.45,6.91
5,Dia,Aceite-de-girasol,1.49,10.99
6,Eroski,Aceite-de-oliva,1.89,64.59
7,Hipercor,Aceite-de-oliva,2.09,154.32
8,Alcampo,Aceite-de-oliva,2.83,62.99
9,Carrefour,Aceite-de-oliva,2.85,63.5


In [49]:
# La función melt de Pandas permite "aplanar" la información, transformándola de un formato ancho a uno largo, que permite a Ploty graficar de manera adecuada.

df_melt = resultado1.melt(
    id_vars=["supermercado", "categoria"], 
    value_vars=["precio_minimo", "precio_maximo"], 
    var_name="tipo_precio", 
    value_name="precio"
)

fig1 = px.bar(
    df_melt, 
    x="supermercado", 
    y="precio", 
    color="categoria", 
    barmode="group",
    facet_col="tipo_precio",  # Crear una columna por tipo de precio (mínimo, máximo)
    labels={
        "supermercado": "Supermercado",
        "precio": "Precio (EUR)",
        "categoria": "Categoría",
        "tipo_precio": "Tipo de Precio"
    },
    title="Comparación de Precios entre Supermercados para cada Categoría de Producto"
)

fig1.update_layout(
    xaxis_title="Supermercado",
    yaxis_title="Precio en EUR",
)
fig1.update_yaxes(matches=None)

fig1.show()

In [28]:
query2 = '''
SELECT 
    h.fecha,
    s.nombre AS supermercado,
    c.nombre AS categoria,
    ROUND(AVG(h.precioeur),2) AS precio_promedio
FROM 
    historico h
JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    h.fecha, s.nombre, c.nombre
ORDER BY 
    c.nombre, s.nombre, h.fecha;
'''

In [29]:
resultado2 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query2)
resultado2

,fecha,supermercado,categoria,precio_promedio
0,2024-01-07,Alcampo,Aceite-de-girasol,2.69
1,2024-01-08,Alcampo,Aceite-de-girasol,2.58
2,2024-01-09,Alcampo,Aceite-de-girasol,2.58
3,2024-01-10,Alcampo,Aceite-de-girasol,2.56
4,2024-02-07,Alcampo,Aceite-de-girasol,2.75
...,...,...,...,...
1939,2024-11-09,Mercadona,Leche,3.79
1940,2024-11-10,Mercadona,Leche,3.70
1941,2024-12-08,Mercadona,Leche,3.64
1942,2024-12-09,Mercadona,Leche,3.79


In [33]:
# Lista de categorías
categorias = resultado2['categoria'].unique()

# Crear subplots: una fila por categoría
fig2 = make_subplots(
    rows=len(categorias), 
    cols=1, 
    shared_xaxes=True, 
    subplot_titles=categorias,
    vertical_spacing=0.1
)

# Añadir una gráfica de línea para cada categoría y supermercado
for i, categoria in enumerate(categorias, start=1):
    # Filtrar los datos para la categoría actual
    df_categoria = resultado2[resultado2['categoria'] == categoria]
    
    for supermercado in df_categoria['supermercado'].unique():
        df_super = df_categoria[df_categoria['supermercado'] == supermercado]
        
        fig2.add_trace(
            go.Scatter(
                x=df_super['fecha'], 
                y=df_super['precio_promedio'], 
                mode='lines',
                name=f"{supermercado} - {categoria}",
                legendgroup=supermercado,  # Agrupa leyendas por supermercado
                line=dict(width=2),
            ),
            row=i, col=1
        )

fig2.update_layout(
    height=300 * 3,  # Ajusta la altura para acomodar todas las subplots
    title="Evolución de precios promedio por categoría ",
    xaxis_title="Fecha",
    yaxis_title="Precio Promedio en EUR",
    showlegend=True
)

fig2.show()

In [73]:
query3 = '''
SELECT 
    h.fecha,
    s.nombre AS supermercado,
    c.nombre AS categoria,
    h.precioeur AS precio_actual,
    h.vareuros AS variacion_euros,
    h.varporcentaje AS variacion_porcentaje
FROM 
    historico h
JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
WHERE 
    h.varporcentaje >= 30 
    OR h.varporcentaje <= -30
ORDER BY 
    c.nombre, s.nombre, h.fecha;
'''

In [74]:
resultado3 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query3)
resultado3

,fecha,supermercado,categoria,precio_actual,variacion_euros,variacion_porcentaje
0,2024-01-08,Alcampo,Aceite-de-girasol,2.19,0.9,69.77
1,2024-06-27,Alcampo,Aceite-de-girasol,2.34,0.59,33.71
2,2024-07-19,Alcampo,Aceite-de-girasol,1.29,-0.9,-41.1
3,2024-09-26,Alcampo,Aceite-de-girasol,2.19,0.8,57.55
4,2024-10-10,Alcampo,Aceite-de-girasol,2.49,0.61,32.45
...,...,...,...,...,...,...
187,2024-09-26,Eroski,Leche,1.42,0.33,30.28
188,2024-09-26,Eroski,Leche,1.42,0.33,30.28
189,2024-10-10,Eroski,Leche,2.81,0.82,41.21
190,2024-07-16,Mercadona,Leche,3.38,-2.33,-40.81


In [75]:
resultado3["precio_actual"] = resultado3["precio_actual"].apply(float)
resultado3["variacion_euros"] = resultado3["variacion_euros"].apply(float)
resultado3["variacion_porcentaje"] = resultado3["variacion_porcentaje"].apply(float)

In [84]:
fig3 = px.area(
    resultado3, 
    x="fecha", 
    y="variacion_porcentaje", 
    color="supermercado", 
    line_group="categoria", 
    labels={
        "fecha": "Fecha",
        "variacion_porcentaje": "Variación (%)",
        "supermercado": "Supermercado",
        "categoria": "Categoría"
    },
    title="Variaciones Inusuales en Precios por Supermercado y Categoría"
)


fig3.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Variación (%)"
)


fig3.show()


In [22]:
query4 = '''
SELECT 
    c.nombre AS categoria,
    ROUND(STDDEV(h.precioeur),2) AS dispersion_precio
FROM 
    historico h
JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
GROUP BY 
    c.nombre
ORDER BY 
    dispersion_precio DESC;
'''

In [23]:
resultado4 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query4)
resultado4

,categoria,dispersion_precio
0,Aceite-de-oliva,16.89
1,Aceite-de-girasol,5.59
2,Leche,2.83


In [20]:
query5 = '''
SELECT 
    s.nombre AS supermercado,
    c.nombre AS categoria,
    ROUND(AVG(h.precioeur),2) AS precio_promedio
FROM 
    historico h
JOIN 
    categorias c
        ON h.id_categoria = c.id_categoria
JOIN 
    supermercados s
        ON h.id_supermercado = s.id_supermercado
GROUP BY 
    s.nombre, c.nombre
ORDER BY 
    c.nombre, precio_promedio;
'''

In [21]:
resultado5 = sf.dbeaver_fetch(sf.dbeaver_conexion("Facua"),query5)
resultado5

,supermercado,categoria,precio_promedio
0,Alcampo,Aceite-de-girasol,2.61
1,Hipercor,Aceite-de-girasol,3.01
2,Eroski,Aceite-de-girasol,3.13
3,Mercadona,Aceite-de-girasol,4.12
4,Dia,Aceite-de-girasol,5.54
5,Carrefour,Aceite-de-girasol,8.23
6,Mercadona,Aceite-de-oliva,13.05
7,Eroski,Aceite-de-oliva,13.26
8,Dia,Aceite-de-oliva,14.72
9,Carrefour,Aceite-de-oliva,17.31


In [60]:
resultado5["precio_promedio"] = resultado5["precio_promedio"].apply(float)

In [85]:
categorias = resultado5['categoria'].unique()
fig5 = sp.make_subplots(rows=len(categorias), cols=1, subplot_titles=categorias, shared_yaxes=False)

for i, categoria in enumerate(categorias):
    df_filtered = resultado5[resultado5['categoria'] == categoria]
    fig5.add_trace(
        go.Bar(x=df_filtered['supermercado'], y=df_filtered['precio_promedio'], name=categoria, text=df_filtered['precio_promedio'], textposition='auto'),
        row=i + 1, col=1
    )

fig5.update_layout(
    barmode='stack',  # Configurar las barras apiladas
    height=300 * 3,
    title_text="Precio Medio de Cada Categoría en Cada Supermercado",
    showlegend=False
)

fig5.show()


In [ ]:
# Cantidad de productos